In [36]:
!pip install kaggle-environments --upgrade -q

In [1]:
from kaggle_environments import make
from scipy.stats import norm

import hyperopt
import numpy as np
import os
if not os.path.exists('tmp'):
    os.mkdir('tmp')
    
base_path = '/Users/sergmiller/Documents/my/bandits'

Loading environment football failed: No module named 'gfootball'


In [2]:
from utils.bandits import Agent, init_agent, \
    compare, gittins_bb_delta, init_template, gittins_with_random, bench

In [3]:
from importlib import reload
import utils
reload(utils)
reload(utils.bandits)
from utils.bandits import exact_gittins

In [4]:
def f(input):
    beta, p, q, eps, drift = input["beta"], input["p"], input["q"], input["eps"], input["drift"]
    gittins_custom = gittins_with_my_and_count_rival_drift.format(
        "beta,p,q,eps,rival_drift={},{},{},{},{}".format(beta, p, q, eps, drift))
    res = compare(Agent(text=gittins_with_random), Agent(text=gittins_custom), T=10)[:3]
    print(input, res)
    return res[1]

In [343]:
compare(Agent(text=gittins_with_count_my), Agent(text=gittins_with_count_my_exp), T=10)[:3]

(0.606474205705885, 6.3, 38.67570296710843)

In [344]:
compare(Agent(text=gittins_with_count_my), Agent(text=gittins_with_count_my_and_rival), T=10)[:3]

(1.2283089875543891e-08, 48.0, 26.649577857819814)

In [347]:
compare(Agent(text=gittins), Agent(text=gittins_with_count_rival_mu), T=10)[:3]

(0.0018893865255212212, 41.7, 42.44066446228192)

In [349]:
compare(Agent(text=gittins), Agent(text=gittins_with_count_rival), T=10)[:3]

(1.1293516115470137e-05, 62.3, 44.86880876510986)

In [362]:
compare(Agent(text=gittins), Agent(text=gittins_with_count_rival_drift), T=10)[:3]

(0.26882824503842084, -11.0, 31.457908385650818)

In [366]:
compare(Agent(text=gittins), Agent(text=gittins_with_my_and_count_rival_drift), T=10)[:3]

(0.47533192912355393, -5.6, 24.80806320533709)

In [404]:
compare(Agent(text=gittins), Agent(text=gittins_with_count_my), T=10)[:3]

(0.00023052734356843025, -29.9, 25.672748197261626)

In [383]:
search_space = {
  "beta": hyperopt.hp.uniform('beta', 0.05, 0.95),
  "p": hyperopt.hp.uniform('p', 0.05, 0.95),
  "q": hyperopt.hp.uniform('q', 0.05, 0.95),
  "eps": hyperopt.hp.uniform('eps', 0, 1e-1),
  "drift": hyperopt.hp.normal('drift', -1e-3, 1e-3),
}

trials = hyperopt.Trials()
best = hyperopt.fmin(
  fn=f,
  space=search_space,
  algo=hyperopt.tpe.suggest,
  max_evals=500,
  verbose=True,
  trials=trials)
print(best)


{'beta': 0.7760118084944925, 'drift': 0.0006101374043450849, 'eps': 0.09216854718415736, 'p': 0.2382199067710455, 'q': 0.7217980278681799}
(0.1782005533239749, -12.9, 30.299999999999997)        
{'beta': 0.8774414143800185, 'drift': 0.0002996220980536783, 'eps': 0.09368269799273073, 'p': 0.5683826438371754, 'q': 0.36799990709395164}
(0.24507897477827856, -9.1, 24.75661527753744)                        
{'beta': 0.06847566165010933, 'drift': -0.0035108516418922444, 'eps': 0.04848637129657493, 'p': 0.06634599968919988, 'q': 0.41298996779010394}
(0.9874736348171315, 0.2, 40.28349537962167)                          
{'beta': 0.16050128960150736, 'drift': -0.0012808303826059284, 'eps': 0.0325453842952085, 'p': 0.7137780515907629, 'q': 0.30161566882141283}
(0.0020578310743403594, -34.2, 35.09358915813542)                     
{'beta': 0.11750555170102675, 'drift': -0.0009281112912669579, 'eps': 0.04141655206174636, 'p': 0.6376868222428131, 'q': 0.9444175581698229}
(0.00012320960339458486, -3

KeyboardInterrupt: 

In [392]:
bench(Agent(text=gittins_bb))

[(2.5255240922780004e-06, 25.9, 17.403735231265728),
 (0.029991876133032967, 23.0, 33.51417610504546),
 (0.32992270880110197, 7.8, 25.317187837514656),
 (0.0001944034405806845, 31.5, 26.7329384841996),
 (0.5074584383928025, 7.0, 33.397604704529336)]

In [396]:
bench(Agent(text=gittins_bb_delta))

[(0.010729090227799871, 38.3, 47.47009584991376),
 (2.9591986278665966e-07, 27.7, 17.088300090997933),
 (0.30022686164525036, 5.2, 15.873247934811575),
 (0.0021999381701743137, 32.6, 33.66957083183568),
 (0.04388293234712658, 21.1, 33.110270309980855)]

In [398]:
a = Agent(text=gittins_bb_delta)
init_agent(a)
a.file

'tmp/b_0.13843056616831617.py'

In [400]:
compare(Agent(file='tmp/b_0.13843056616831617.py'), Agent(text=gittins))

(0.008097833972786038,
 28.2,
 33.67729205265768,
 array([554., 652., 556., 634., 625., 627., 597., 781., 625., 644.]),
 array([548., 621., 590., 589., 568., 596., 603., 686., 607., 605.]))

In [407]:
a = Agent(text=gittins_with_count_my)
init_agent(a)
a.file

'tmp/b_0.36080881811064225.py'

In [410]:
compare(Agent(file='tmp/b_0.8473257048600334.py'), Agent(text=gittins))

(0.07969100837095777,
 12.3,
 22.194819215303376,
 array([659., 653., 607., 647., 633., 586., 626., 608., 586., 592.]),
 array([609., 627., 614., 612., 600., 595., 638., 589., 581., 609.]))

In [411]:
compare(Agent(text=gittins_with_count_my), Agent(text=gittins))

(0.010155020119002756,
 20.6,
 25.342454498331453,
 array([546., 581., 608., 615., 599., 609., 591., 684., 651., 691.]),
 array([573., 547., 602., 575., 569., 600., 609., 636., 611., 647.]))

In [409]:
compare(Agent(file='tmp/b_0.36080881811064225.py'), Agent(text=gittins))

(3.4674259775474264e-05,
 26.5,
 20.239812252093643,
 array([586., 649., 630., 600., 649., 673., 563., 627., 616., 629.]),
 array([577., 637., 608., 561., 638., 634., 542., 550., 610., 600.]))

In [412]:
compare(Agent(file='tmp/b_0.36080881811064225.py'), Agent(text=gittins), T=20)

(0.006451355673915332,
 16.25,
 26.679345944006947,
 array([638., 570., 647., 660., 671., 626., 682., 592., 634., 663., 605.,
        651., 616., 628., 597., 620., 643., 582., 617., 582.]),
 array([621., 593., 610., 631., 683., 591., 621., 560., 669., 615., 598.,
        591., 591., 612., 587., 631., 622., 559., 602., 612.]))

In [415]:
compare(Agent(file='kernels/multiarmed_bandit_agent.py'), Agent(text=gittins), T=20)

(0.0007908909690796149,
 20.9,
 27.851211822827384,
 array([664., 690., 587., 618., 669., 526., 611., 661., 642., 615., 560.,
        618., 659., 626., 599., 680., 632., 641., 627., 654.]),
 array([640., 672., 558., 593., 614., 565., 618., 644., 612., 587., 577.,
        612., 637., 565., 614., 602., 634., 619., 569., 629.]))

In [414]:
compare(Agent(file='kernels/softmax_ucb.py'), Agent(text=gittins), T=20)

(0.7089664212117914,
 2.35,
 28.157192686771882,
 array([628., 627., 590., 582., 598., 568., 611., 657., 650., 585., 710.,
        543., 657., 676., 566., 659., 626., 700., 661., 612.]),
 array([620., 635., 585., 572., 609., 577., 613., 619., 593., 607., 666.,
        589., 663., 668., 583., 657., 632., 660., 722., 589.]))

In [416]:
compare(Agent(file='kernels/upper_confidence.py'), Agent(text=gittins), T=20)

(4.9853040265783087e-64,
 -123.45,
 32.679466029909364,
 array([520., 548., 546., 530., 515., 520., 627., 597., 551., 540., 537.,
        510., 563., 560., 525., 604., 500., 547., 589., 497.]),
 array([645., 683., 615., 677., 655., 640., 689., 735., 676., 649., 650.,
        711., 686., 701., 691., 673., 650., 642., 694., 633.]))

In [417]:
compare(Agent(file='kernels/multiarmed_bandit_agent.py'), Agent(text=gittins_with_random), T=20)

(1.4081626254579164e-05,
 33.05,
 34.03597361616089,
 array([663., 670., 649., 650., 682., 681., 578., 685., 663., 626., 629.,
        561., 641., 521., 643., 704., 668., 582., 641., 709.]),
 array([594., 596., 659., 602., 603., 621., 567., 676., 601., 583., 636.,
        535., 585., 576., 601., 685., 628., 593., 600., 644.]))

In [418]:
compare(Agent(file='kernels/softmax_ucb.py'), Agent(text=gittins_with_random), T=20)

(0.04821592751221526,
 14.3,
 32.37298256262466,
 array([583., 641., 584., 666., 636., 590., 639., 607., 633., 626., 618.,
        648., 584., 673., 666., 613., 590., 666., 579., 634.]),
 array([597., 584., 582., 659., 587., 619., 642., 604., 558., 620., 618.,
        589., 586., 661., 652., 615., 644., 617., 581., 575.]))

In [422]:
bench(Agent(text=gittins_bb_delta))

[(4.2854171320643786e-08, 54.1, 31.226431112120384),
 (2.0465870932658617e-24, 41.4, 12.839003076563227),
 (0.9525250069252739, 0.6, 31.86910729844813),
 (0.0009244556884093576, 37.0, 35.32138162643132),
 (0.9779150724415098, 0.2, 22.846443924602355),
 (0.13794316903592332, -11.3, 24.087548650703333),
 (0.23564468912492498, 12.4, 33.064180014027265),
 (1.1411492310898283e-22, 34.0, 10.972693379476162),
 (0.3037168800018688, -9.6, 29.516774891576485),
 (6.083455761527968e-52, 128.5, 26.796454989419775)]

In [428]:
import torch
from torch import nn

In [426]:
!pip install torch

     |████████████████████████████████| 110.0 MB 5.3 MB/s eta 0:00:01    |████████████                    | 41.4 MB 3.2 MB/s eta 0:00:22


In [572]:
# neural_with_lr = neural.format("lr = {}".format(1e-3))
# hidden = 16
compare(Agent(text=neural), Agent(text=gittins_with_random), T=20)

(0.04817399799390361,
 -33.05,
 74.80606593051127,
 array([628., 570., 670., 548., 514., 589., 615., 616., 643., 499., 603.,
        449., 652., 576., 573., 673., 551., 636., 523., 672.]),
 array([655., 605., 651., 742., 648., 574., 597., 574., 600., 572., 629.,
        661., 674., 571., 613., 661., 616., 582., 616., 620.]))

In [575]:
# hidden=32
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.3143794305537898,
 -16.1,
 50.60523688315272,
 array([619., 631., 592., 612., 513., 606., 672., 666., 640., 595.]),
 array([622., 631., 583., 654., 590., 722., 639., 644., 582., 640.]))

In [583]:
# hidden=64
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.0006475889972293117,
 -41.4,
 38.382808651790974,
 array([594., 522., 606., 585., 577., 531., 588., 593., 586., 619.]),
 array([613., 612., 616., 553., 629., 606., 628., 675., 661., 622.]))

In [581]:
# hidden=128
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.0004958468418106363,
 -31.1,
 28.236324123369883,
 array([538., 601., 656., 545., 593., 587., 570., 539., 598., 601.]),
 array([582., 643., 641., 586., 611., 639., 627., 522., 615., 673.]))

In [585]:
# hidden=64, without layernorm
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.10396468619078034,
 -23.9,
 46.483222779837455,
 array([537., 566., 619., 623., 615., 661., 625., 635., 615., 580.]),
 array([649., 657., 660., 650., 596., 612., 622., 671., 619., 579.]))

In [591]:
# hidden=64, without layernorm with gittins baseline
compare(Agent(text=neural), Agent(text=gittins_with_random), T=20)

(0.000383703897760926,
 29.3,
 36.9,
 array([660., 655., 614., 553., 627., 626., 698., 666., 653., 713., 666.,
        619., 628., 668., 608., 603., 611., 656., 618., 633.]),
 array([643., 592., 563., 573., 572., 569., 679., 588., 604., 643., 620.,
        583., 678., 635., 588., 564., 677., 598., 603., 617.]))

In [593]:
# hidden=128, without layernorm with gittins baseline
compare(Agent(text=neural), Agent(text=gittins_with_random), T=20)

(5.259240510589902e-06,
 27.1,
 26.6118394704312,
 array([625., 587., 626., 607., 604., 724., 592., 650., 647., 605., 643.,
        602., 625., 580., 660., 609., 646., 623., 712., 586.]),
 array([590., 536., 573., 572., 553., 647., 571., 666., 636., 589., 561.,
        557., 599., 553., 652., 625., 628., 616., 691., 596.]))

In [597]:
# hidden=128, without layernorm with gittins baseline, 3 layers
compare(Agent(text=neural), Agent(text=gittins_with_random), T=20)

(0.0009306281273881362,
 23.85,
 32.216882220351486,
 array([567., 668., 647., 606., 626., 607., 635., 653., 612., 548., 673.,
        728., 636., 562., 565., 603., 689., 553., 559., 578.]),
 array([567., 634., 618., 589., 610., 630., 583., 568., 611., 560., 630.,
        662., 610., 509., 629., 564., 656., 518., 524., 566.]))

In [598]:
# hidden=128, without layernorm with gittins baseline, 3 layers, sigmoid
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.006920246178702476,
 18.5,
 21.66217902243447,
 array([658., 629., 578., 635., 577., 628., 650., 667., 619., 643.]),
 array([665., 596., 560., 604., 595., 601., 593., 646., 589., 650.]))

In [600]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.004622009254888536,
 31.0,
 34.61213659975356,
 array([738., 562., 621., 636., 679., 619., 682., 658., 685., 588.]),
 array([653., 556., 624., 595., 597., 647., 673., 631., 630., 552.]))

In [612]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.009804144836376547,
 16.0,
 19.590814173994914,
 array([630., 694., 586., 618., 605., 634., 661., 689., 657., 666.]),
 array([589., 669., 615., 603., 607., 632., 639., 658., 635., 633.]))

In [608]:
# hidden=256, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.2261938354909282,
 13.1,
 34.22995763947131,
 array([588., 599., 711., 651., 601., 552., 591., 658., 577., 616.]),
 array([606., 576., 692., 605., 629., 574., 615., 615., 498., 603.]))

In [610]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves, lr=1e-2
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(8.173946298618923e-06,
 -149.1,
 105.7028381832768,
 array([665., 473., 439., 643., 449., 555., 651., 499., 448., 547.]),
 array([686., 705., 647., 582., 701., 694., 726., 686., 749., 684.]))

In [616]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves(my + rival)
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.07436803382292292,
 10.3,
 18.254040648579696,
 array([558., 592., 669., 614., 709., 609., 622., 609., 628., 648.]),
 array([553., 575., 668., 610., 691., 588., 654., 600., 584., 632.]))

In [620]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves(my + rival) + rewards
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.0013064904372689834,
 31.0,
 30.495901363953813,
 array([634., 577., 663., 643., 631., 688., 656., 680., 645., 683.]),
 array([655., 567., 605., 584., 636., 665., 590., 647., 631., 610.]))

In [622]:
# hidden=128, without layernorm with gittins baseline, 3 layers, sigmoid, 5 last moves(my + rival) + rewards
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.02036659303085346,
 -23.9,
 32.58358482426389,
 array([652., 599., 618., 551., 609., 600., 642., 637., 570., 675.]),
 array([651., 661., 590., 620., 599., 611., 658., 651., 641., 710.]))

In [624]:
# hidden=128, without layernorm with gittins baseline, 3 layers, sigmoid, 5 last moves(my + rival) + rewards,lr = 1e-2
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.3570434791327334,
 -9.3,
 31.931332574761107,
 array([686., 651., 591., 652., 548., 634., 655., 680., 597., 562.]),
 array([683., 705., 610., 682., 566., 632., 684., 613., 581., 593.]))

In [626]:
# hidden=256, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves(my + rival) + rewards
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.006018923380864479,
 19.9,
 22.91047795223836,
 array([628., 630., 692., 607., 652., 637., 674., 696., 654., 622.]),
 array([600., 639., 661., 614., 623., 643., 678., 652., 617., 566.]))

In [628]:
# hidden=512, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves(my + rival) + rewards
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.7230291358409504,
 3.5,
 31.228992939254383,
 array([545., 609., 594., 651., 616., 564., 715., 640., 664., 637.]),
 array([527., 594., 595., 658., 639., 623., 693., 621., 599., 651.]))

In [630]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves(my + rival) + rewards -softsign
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.8955202509678022,
 -1.9,
 45.752486271239945,
 array([604., 631., 575., 604., 559., 605., 589., 662., 572., 650.]),
 array([688., 588., 577., 619., 594., 556., 581., 628., 639., 600.]))

In [633]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves(my + rival) + rewards, mse loss
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.4814618952066887,
 -7.0,
 31.445190411253673,
 array([581., 602., 568., 538., 566., 672., 619., 634., 595., 669.]),
 array([579., 575., 587., 567., 609., 641., 649., 666., 625., 616.]))

In [635]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves(my + rival) + rewards, mse loss,lr=1e-2
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(1.3027992858897352e-09,
 -100.8,
 52.539128275981135,
 array([513., 508., 629., 538., 609., 560., 569., 473., 527., 545.]),
 array([649., 571., 691., 557., 670., 654., 730., 678., 613., 666.]))

In [637]:
# hidden=128, without layernorm with gittins baseline, 2 layers, sigmoid, 5 last moves(my + rival) + rewards (again)
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.07408167061938936,
 15.6,
 27.619558287561368,
 array([670., 618., 603., 668., 594., 609., 595., 602., 718., 635.]),
 array([627., 627., 598., 674., 568., 647., 592., 567., 667., 589.]))

In [645]:
a = Agent(text=neural)
init_agent(a)
a.file

'tmp/b_0.4204600914015785.py'

In [649]:
res = bench(Agent(file='tmp/b_0.4204600914015785.py'))

(0.3972262309976057, 9.7, 36.232720019341635)
(0.03945976742325765, 16.7, 25.64390765854533)
(0.008419059869329705, -28.4, 34.08577415873079)
(0.4787347617112808, 6.9, 30.804058174208148)
(0.0009292589432535868, -24.4, 23.303218661807215)
(3.6377097512975197e-07, -40.5, 25.17637781731121)
(0.0001163659494051545, -23.6, 19.365949499056327)
(0.6908953278170049, 3.7, 29.42464953062313)
(0.04314162037527695, -23.3, 36.43363830308469)
(5.835892518606307e-30, 136.3, 37.90527667752868)


In [651]:
compare(Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/ucb_decay.py'), Agent(text=gittins_with_random))

(0.15744713612524386,
 -13.4,
 29.973988723558296,
 array([633., 550., 509., 646., 593., 638., 633., 588., 630., 577.]),
 array([627., 579., 597., 651., 621., 661., 610., 594., 614., 577.]))

In [652]:
compare(Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/bayesian_ucb.py'), Agent(text=gittins_with_random))

(1.5001900765283778e-05,
 52.3,
 38.207460004559316,
 array([735., 668., 624., 660., 659., 674., 607., 565., 650., 650.]),
 array([623., 600., 584., 579., 698., 631., 582., 498., 577., 597.]))

In [653]:
compare(Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/thompson.py'), Agent(text=gittins_with_random))

(0.015175617949015455,
 25.4,
 33.07929866245655,
 array([653., 614., 607., 643., 636., 643., 664., 571., 643., 657.]),
 array([629., 590., 656., 585., 571., 578., 641., 580., 619., 628.]))

In [666]:
# hidden=128, without layernorm with gittins delta_bb baseline, 2 layers, sigmoid, 5 last moves(my + rival) + rewards (again)
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.0045348001696681964,
 34.9,
 38.88302971734584,
 array([670., 647., 588., 611., 739., 669., 662., 636., 677., 634.]),
 array([621., 594., 578., 629., 624., 636., 657., 609., 596., 640.]))

In [667]:
compare(Agent(text=neural), Agent(text=gittins_bb_delta), T=20)

(0.4150601933334106,
 -5.05,
 27.710061349625338,
 array([618., 552., 648., 582., 655., 640., 595., 628., 644., 623., 567.,
        666., 631., 608., 621., 630., 650., 630., 575., 590.]),
 array([647., 555., 613., 565., 618., 645., 622., 604., 649., 614., 622.,
        658., 621., 666., 658., 641., 688., 599., 566., 603.]))

In [672]:
# gittins delta_bb baseline, 1 layer, sigmoid, 5 last moves(my + rival) + rewards (again)
compare(Agent(text=neural), Agent(text=gittins_bb_delta), T=20)

(0.9650311781006329,
 0.3,
 30.602450882241445,
 array([644., 587., 580., 579., 567., 580., 693., 613., 615., 540., 507.,
        696., 671., 671., 694., 575., 622., 658., 578., 664.]),
 array([655., 636., 602., 598., 582., 602., 700., 561., 589., 539., 496.,
        654., 614., 683., 661., 600., 675., 627., 570., 684.]))

In [670]:
# gittins delta_bb baseline, 1 layer, sigmoid, 5 last moves(my + rival) + rewards (again)
compare(Agent(text=neural), Agent(text=gittins_with_random), T=10)

(0.0005995478898804283,
 -39.1,
 36.02901608426186,
 array([630., 603., 557., 527., 579., 562., 562., 607., 564., 537.]),
 array([615., 602., 606., 630., 654., 644., 581., 628., 603., 556.]))

In [673]:
a = Agent(text=neural)
init_agent(a)
a.file

'tmp/b_0.31264246571420906.py'

In [674]:
res = bench(Agent(file='tmp/b_0.31264246571420906.py'))

['gittins', (2.1475984606729273e-06, 56.2, 37.50146663798631)]
['gittins_with_random', (1.3715233030447107e-06, 34.4, 22.526428922490133)]
['gittins_with_count_my', (0.0003151042021039252, 19.1, 16.76573887426379)]
['gittins_with_count_rival_drift', (1.6838757781366005e-10, 47.1, 23.317161062187655)]
['gittins_with_my_and_count_rival_drift', (0.11963707274256954, 18.5, 37.590557324945316)]
['gittins_bb', (0.773553859300014, 3.6, 39.565641660410364)]
['gittins_bb_delta', (0.4212070972968093, 7.4, 29.093641917092473)]
['softmax_ucb', (0.002660131961789012, 20.9, 21.9974998579384)]
['multiarmed_bandit_agent', (0.04146487952384929, 11.8, 18.301912468373352)]
['upper_confidence', (7.38117376486062e-30, 127.4, 35.494224882366424)]
['ucb_decay', (1.3001477544135825e-06, 38.7, 25.28655769376291)]
['bayesian_ucb', (0.6258482932879363, -5.0, 32.428382630035685)]
['thompson', (0.7459054527070903, 1.8, 17.565876010037186)]


In [675]:
compare(Agent(text=neural), Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/bayesian_ucb.py'), T=20)

(0.17912212685919382,
 -11.0,
 36.616935972306585,
 array([624., 691., 663., 656., 519., 600., 593., 612., 714., 582., 628.,
        616., 609., 618., 576., 573., 698., 630., 563., 672.]),
 array([619., 632., 700., 619., 571., 639., 617., 617., 667., 637., 653.,
        672., 611., 653., 545., 612., 690., 678., 597., 628.]))

In [676]:
compare(Agent(text=gittins_bb_delta), Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/bayesian_ucb.py'), T=20)

(0.09443938833273353,
 -10.45,
 27.943648652242967,
 array([636., 618., 515., 603., 634., 565., 588., 663., 689., 616., 563.,
        610., 586., 644., 596., 596., 634., 586., 595., 618.]),
 array([678., 633., 524., 613., 667., 572., 589., 628., 637., 620., 563.,
        643., 576., 654., 584., 670., 634., 629., 637., 613.]))

In [684]:
# compare(Agent(text=gittins_bb_delta), Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/max_likelihood.py'), T=10)

In [682]:
compare(Agent(text=gittins_bb_delta), Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/optimized_ucb.py'), T=10)

<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide


(1.4805707294350223e-25,
 80.8,
 24.453220646777797,
 array([633., 628., 656., 625., 675., 702., 666., 719., 664., 670.]),
 array([568., 550., 553., 595., 570., 614., 572., 624., 617., 567.]))

In [5]:
compare(Agent(text=gittins_bb_delta), 
        Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/optimized_ucb.py'), T=10)

<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide
<string>:82: RuntimeWarning: invalid value encountered in true_divide


(2.779122634987333e-05,
 42.2,
 31.842738575694145,
 array([550., 629., 626., 680., 588., 649., 666., 599., 587., 552.]),
 array([564., 576., 575., 616., 573., 551., 620., 524., 563., 542.]))

In [28]:
def f_neural(input):
    input['hidden'] = int(input['hidden'])
    print(input)
    neural_custom = init_template(neural, input)
    res = compare(Agent(text=neural), Agent(text=neural_custom), T=5)[:3]
    print(res[1] + res[2], res)
    return res[1] + res[2]  # since res[1] < 0

In [29]:
search_space = {
  "beta": hyperopt.hp.uniform('beta', 0.01, 0.99),
  "p": hyperopt.hp.uniform('p', 0.01, 0.99),
  "q": hyperopt.hp.uniform('q', 0.01, 0.99),
  "eps": hyperopt.hp.loguniform('eps', np.log(1e-5), np.log(1e-1)),
  "rival_drift": hyperopt.hp.normal('rival_drift', -1e-3, 1e-3),
  "lr": hyperopt.hp.loguniform('lr', np.log(1e-3), np.log(1e-2)),
  "decay": hyperopt.hp.normal('decay', -1e-1, 1e-1),
  "hidden": hyperopt.hp.loguniform('hidden', np.log(32), np.log(128))
}

trials = hyperopt.Trials()
best = hyperopt.fmin(
  fn=f_neural,
  space=search_space,
  algo=hyperopt.tpe.suggest,
  max_evals=500,
  verbose=True,
  trials=trials)
print(best)


{'beta': 0.48527819163912284, 'decay': -0.1219426368279326, 'eps': 0.00017302765687648595, 'hidden': 123, 'lr': 0.006003122670992075, 'p': 0.1608509472229554, 'q': 0.2788704008002208, 'rival_drift': -0.0021116617617665725}
832.91879273381                                        
(7.416282814338792e-08, 588.4, 244.51879273381013)     
{'beta': 0.4664554497852051, 'decay': -0.17864610808752782, 'eps': 0.0005829038634554092, 'hidden': 69, 'lr': 0.004202915019875021, 'p': 0.5585278605405385, 'q': 0.5229545570767066, 'rival_drift': -0.0007344196824117082}
846.9784938665006                                                               
(0.0, 825.6, 21.37849386650051)                                                 
{'beta': 0.3151195214058192, 'decay': 0.009754473651137269, 'eps': 0.0022345918449839172, 'hidden': 60, 'lr': 0.007794848273784913, 'p': 0.20600293420977916, 'q': 0.18686875356747387, 'rival_drift': 0.0005290310881097614}
19.150796164885804                                          

KeyboardInterrupt: 

In [5]:
# {'beta': 0.23349050923773682, 'decay': 0.012241718750925756, 'eps': 1.953962396646424e-05, 'hidden': 63, 'lr': 0.0013236578179036044, 'p': 0.828200914946154, 'q': 0.35291732447593555, 'rival_drift': 0.0005123995896210666}
# -8.489624672592807                                                                     
# (0.003155782796970563, -35.0, 26.510375327407193)  

# {'beta': 0.436592528014877, 'decay': 0.02040693745330466, 'eps': 9.101721194641989e-05, 'hidden': 98, 'lr': 0.0010510764254283399, 'p': 0.4922803750654748, 'q': 0.7962969377374626, 'rival_drift': 0.0007186636650624174}
# -10.527115509800886                                                                    
# (0.00038070475398410594, -28.4, 17.872884490199112)  

neural_bb_delta_params = {
    'beta': 0.41069163851066504, 
                          'decay': 0.02999671380157681, 
                   'eps': 4.906904712248718e-05,
                    'hidden': 107, 
                          'lr': 0.0014347728372142037, 
                   'p': 0.6199112732964978, 
                          'q': 0.7239785022098951,
                   'rival_drift': 0.0007989760655281166}
# -23.728872566181053                                                                     
# (1.9020755301043156e-17, -32.2, 8.471127433818948) 

In [6]:
a = Agent(text=init_template(neural, neural_bb_delta_params))
init_agent(a)
a.file

'tmp/b_0.5841452101653711.py'

In [7]:
bench(a)

['gittins', (2.3990640074756468e-05, 31.2, 23.357225862674703)]
['gittins_with_random', (0.00013899825238214027, 51.1, 42.41332337839137)]
['gittins_with_count_my', (0.3813837129815032, 10.5, 37.9321763151022)]
['gittins_with_count_rival_drift', (1.8716005378351934e-09, 41.8, 21.999090890307265)]
['gittins_with_my_and_count_rival_drift', (0.9787103324825969, -0.3, 35.55010548507557)]
['gittins_bb', (0.6441258290401195, 4.7, 32.17467948558307)]
['gittins_bb_delta', (0.6170685672027434, -5.2, 32.88707952980927)]
['softmax_ucb', (5.453728754321697e-06, 31.9, 22.18760915466108)]
['multiarmed_bandit_agent', (0.07515785302276345, 13.4, 23.812601705819546)]
['upper_confidence', (4.976225476678012e-67, 138.4, 25.30296425322535)]
['ucb_decay', (1.1236608347786399e-06, 40.0, 25.98076211353316)]
['bayesian_ucb', (0.07619129110113643, -12.2, 21.756838005555863)]
['thompson', (0.003841505072188575, 23.9, 26.143641674411008)]
['neural', (0.5845754900275143, 4.9, 28.342371107583784)]
<string>:82: Run

[['gittins', (2.3990640074756468e-05, 31.2, 23.357225862674703)],
 ['gittins_with_random', (0.00013899825238214027, 51.1, 42.41332337839137)],
 ['gittins_with_count_my', (0.3813837129815032, 10.5, 37.9321763151022)],
 ['gittins_with_count_rival_drift',
  (1.8716005378351934e-09, 41.8, 21.999090890307265)],
 ['gittins_with_my_and_count_rival_drift',
  (0.9787103324825969, -0.3, 35.55010548507557)],
 ['gittins_bb', (0.6441258290401195, 4.7, 32.17467948558307)],
 ['gittins_bb_delta', (0.6170685672027434, -5.2, 32.88707952980927)],
 ['softmax_ucb', (5.453728754321697e-06, 31.9, 22.18760915466108)],
 ['multiarmed_bandit_agent', (0.07515785302276345, 13.4, 23.812601705819546)],
 ['upper_confidence', (4.976225476678012e-67, 138.4, 25.30296425322535)],
 ['ucb_decay', (1.1236608347786399e-06, 40.0, 25.98076211353316)],
 ['bayesian_ucb', (0.07619129110113643, -12.2, 21.756838005555863)],
 ['thompson', (0.003841505072188575, 23.9, 26.143641674411008)],
 ['neural', (0.5845754900275143, 4.9, 28.342

In [12]:
np.random.randint(low=0,high=10,size=(2,5))

array([[0, 6, 6, 4, 7],
       [2, 4, 2, 9, 2]])

In [6]:
a = Agent(text=utils.bandits.neural)
init_agent(a)
print(a.file)

tmp/b_0.30842465322336077.py


In [14]:
compare(
        Agent(text=utils.bandits.neural.format("use_reinforce,eps,lr=True,0.03,1e-4", "")),
        Agent(text=utils.bandits.neural)
)

(5.763472200361243e-76,
 -116.9,
 20.042205467462907,
 array([574., 522., 521., 617., 559., 567., 471., 587., 578., 583.]),
 array([687., 639., 629., 761., 688., 651., 621., 698., 702., 672.]))

In [15]:
compare(
        Agent(text=utils.bandits.neural.format("use_reinforce,batch,lr,eps=True,4,1e-4,0.03", "baseline=exact_gittins")),
        Agent(text=utils.bandits.neural)
) 

(3.4894708609258446e-17,
 -68.5,
 25.699221778100597,
 array([670., 590., 566., 554., 636., 577., 591., 588., 636., 654.]),
 array([705., 630., 653., 641., 755., 668., 653., 655., 691., 696.]))

In [20]:
compare(
        Agent(text=utils.bandits.neural.format("use_reinforce,batch,lr,eps=True,4,1e-4,0.03", "baseline=exact_gittins")),
        Agent(text=utils.bandits.exact_gittins)
) 

(1.87292138812188e-67,
 -92.1,
 16.783622970026464,
 array([495., 585., 534., 601., 525., 597., 631., 635., 551., 611.]),
 array([614., 656., 656., 681., 626., 689., 715., 715., 626., 708.]))

In [17]:
compare(
        Agent(text=utils.bandits.neural),
        Agent(text=utils.bandits.exact_gittins)
) 

(0.09041480660032127,
 10.0,
 18.676188047886004,
 array([604., 621., 643., 610., 588., 623., 677., 663., 689., 648.]),
 array([599., 647., 605., 617., 574., 609., 683., 634., 681., 617.]))

In [23]:
compare(
        Agent(text=utils.bandits.neural), 
        Agent(text=utils.bandits.exact_gittins)
) 

(0.34202057750166037,
 -5.1,
 16.973214191778762,
 array([631., 621., 650., 659., 628., 647., 669., 598., 613., 607.]),
 array([660., 617., 627., 652., 650., 643., 679., 597., 646., 603.]))

In [25]:
compare(
        Agent(text=utils.bandits.neural),
        Agent(text=utils.bandits.exact_gittins)
)

(0.44671148393834237,
 5.6,
 23.273160507331188,
 array([667., 596., 620., 607., 573., 601., 654., 649., 656., 664.]),
 array([641., 613., 612., 590., 550., 630., 636., 642., 618., 699.]))

In [33]:
#add thompson decay feature
compare(
        Agent(text=utils.bandits.neural),
        Agent(text=utils.bandits.exact_gittins)
)

(0.2939844778871168,
 8.2,
 24.7095123383688,
 array([635., 650., 730., 596., 619., 600., 576., 683., 609., 568.]),
 array([669., 603., 706., 589., 613., 629., 553., 646., 611., 565.]))

In [5]:
compare(
        Agent(text=utils.bandits.neural.format("use_reinforce,batch,lr=True,16,1e-4", "baseline=exact_gittins")),
        Agent(text=utils.bandits.neural)
) 

(0.0012114718263648368,
 -37.0,
 36.155220923125334,
 array([574., 693., 548., 627., 645., 537., 615., 619., 601., 596.]),
 array([673., 741., 631., 629., 632., 605., 628., 668., 602., 616.]))

In [22]:
print(utils.bandits.neural)

import numpy as np
import torch
from torch import nn
import torch.optim as optim

from scipy.stats import beta

from abc import ABCMeta, abstractmethod

class AbstractAgent(metaclass=ABCMeta):   
    def init_actions(self, n_actions):
        self._successes = np.zeros(n_actions)
        self._failures = np.zeros(n_actions)
        self._rival_moves = np.zeros(n_actions)
        self._total_pulls = 0
    
    @abstractmethod
    def get_action(self):
        '''
        Get current best action
        :rtype: int
        '''
        pass
    
    def update(self, action, reward, rival_move=None):
        '''
        Observe reward from action and update agent's internal parameters
        :type action: int
        :type reward: int
        '''
        self._total_pulls += 1
        if reward > 0:
            self._successes[action] += 1
        else:
            self._failures[action] += 1
            
        if rival_move is not None:
            self._rival_moves[rival_move] += 1
  

In [36]:
a = Agent(text=utils.bandits.neural)
init_agent(a)
print(a.file)
bench(a)

tmp/b_0.3255779459099547.py
['gittins', (0.007620852577511365, 24.1, 28.56028711340276)]
['gittins_with_random', (6.126078373502434e-05, 34.3, 27.063074474272135)]
['gittins_with_count_my', (0.4139638903491465, 7.2, 27.870414421030773)]
['gittins_with_count_rival_drift', (1.3499562219649051e-08, 50.0, 27.83882181415011)]
['gittins_with_my_and_count_rival_drift', (0.022753895504997446, 16.5, 22.90960497258737)]
['gittins_bb', (0.31340391253657407, 9.6, 30.11378421919105)]
['gittins_bb_delta', (0.38994074165323456, 8.4, 30.897249068485046)]
['softmax_ucb', (0.0014427798851282295, 26.4, 26.203816515919964)]
['multiarmed_bandit_agent', (0.7142810604668595, -3.2, 27.639826338094093)]
['upper_confidence', (2.3558001371656748e-42, 153.1, 35.49774640734254)]
['ucb_decay', (0.0277111528427185, 16.6, 23.84617369726221)]
['bayesian_ucb', (0.9851096187670044, -0.2, 33.88746080779733)]
['thompson', (0.42182948903103745, -9.0, 35.43162429243119)]
['neural', (0.8989731090214599, 1.1, 27.3987225979606

[['gittins', (0.007620852577511365, 24.1, 28.56028711340276)],
 ['gittins_with_random', (6.126078373502434e-05, 34.3, 27.063074474272135)],
 ['gittins_with_count_my', (0.4139638903491465, 7.2, 27.870414421030773)],
 ['gittins_with_count_rival_drift',
  (1.3499562219649051e-08, 50.0, 27.83882181415011)],
 ['gittins_with_my_and_count_rival_drift',
  (0.022753895504997446, 16.5, 22.90960497258737)],
 ['gittins_bb', (0.31340391253657407, 9.6, 30.11378421919105)],
 ['gittins_bb_delta', (0.38994074165323456, 8.4, 30.897249068485046)],
 ['softmax_ucb', (0.0014427798851282295, 26.4, 26.203816515919964)],
 ['multiarmed_bandit_agent', (0.7142810604668595, -3.2, 27.639826338094093)],
 ['upper_confidence', (2.3558001371656748e-42, 153.1, 35.49774640734254)],
 ['ucb_decay', (0.0277111528427185, 16.6, 23.84617369726221)],
 ['bayesian_ucb', (0.9851096187670044, -0.2, 33.88746080779733)],
 ['thompson', (0.42182948903103745, -9.0, 35.43162429243119)],
 ['neural', (0.8989731090214599, 1.1, 27.3987225979

In [71]:
import torch
from torch import nn

In [72]:
torch.random.manual_seed(1)
# np.random.seed(1)

In [68]:
model = nn.Sequential(
    nn.Softsign(),
    nn.Linear(4, 5),
    nn.Sigmoid(),
    nn.Linear(5, 1)
)

In [69]:
model[1].weight

Parameter containing:
tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247],
        [ 0.1826, -0.1949, -0.0365, -0.0450],
        [ 0.0725, -0.0020,  0.4371,  0.1556]], requires_grad=True)

In [63]:
model[1].weight

Parameter containing:
tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247],
        [ 0.1826, -0.1949, -0.0365, -0.0450],
        [ 0.0725, -0.0020,  0.4371,  0.1556]], requires_grad=True)

In [66]:
model[1].weight

Parameter containing:
tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247],
        [ 0.1826, -0.1949, -0.0365, -0.0450],
        [ 0.0725, -0.0020,  0.4371,  0.1556]], requires_grad=True)

In [75]:
import tqdm

In [81]:
for i in tqdm.tqdm(np.arange(20)):
    print(i)
    print(compare(
        Agent(text=utils.bandits.neural.format("seed={}".format(i), "")),
        Agent(text=utils.bandits.neural)
    ))

  0%|          | 0/20 [00:00<?, ?it/s]

0


  5%|▌         | 1/20 [02:04<39:23, 124.40s/it]

(0.10610246697284985, 13.9, 27.20091910211859, array([608., 623., 540., 679., 648., 647., 601., 673., 639., 607.]), array([645., 609., 516., 668., 647., 618., 545., 634., 666., 578.]))
1


 10%|█         | 2/20 [04:13<37:44, 125.79s/it]

...

(0.26565483278347923, 12.5, 35.51126581804709, array([613., 588., 585., 714., 586., 638., 674., 618., 666., 729.]), array([615., 594., 577., 643., 593., 665., 595., 624., 628., 752.]))
14


 75%|███████▌  | 15/20 [1:59:36<31:53, 382.61s/it]

(0.901423471843342, -1.0, 25.53037406698147, array([573., 682., 631., 618., 642., 604., 611., 608., 670., 694.]), array([582., 645., 642., 607., 661., 640., 623., 637., 657., 649.]))
15


 75%|███████▌  | 15/20 [2:01:38<40:32, 486.57s/it]


KeyboardInterrupt: 

In [82]:
compare(
        Agent(text=utils.bandits.neural.format("seed={}".format(0), "")),
        Agent(text=utils.bandits.neural),T=20
)

(0.0035838829703721377,
 -16.6,
 25.488036409264645,
 array([630., 628., 638., 578., 642., 648., 651., 692., 674., 651., 577.,
        598., 590., 579., 663., 566., 623., 621., 614., 681.]),
 array([638., 603., 684., 626., 663., 674., 698., 693., 638., 649., 595.,
        602., 627., 635., 688., 586., 629., 622., 664., 662.]))

In [11]:
compare(
        Agent(text=utils.bandits.neural.format("use_sep_nn=True", "")),
        Agent(text=utils.bandits.neural)
)

(0.551068675364717,
 3.3,
 17.504570831642805,
 0.6,
 array([678., 725., 601., 652., 645., 622., 578., 562., 592., 602.]),
 array([678., 679., 619., 658., 641., 614., 595., 562., 574., 604.]))

In [12]:
compare(
        Agent(text=utils.bandits.neural.format("use_sep_nn,sep_components=True,16", "")),
        Agent(text=utils.bandits.neural)
)

(0.6028367599019422,
 -5.0,
 30.38749742904144,
 0.4,
 array([677., 622., 624., 620., 631., 650., 627., 581., 603., 581.]),
 array([640., 643., 642., 658., 619., 608., 599., 616., 618., 623.]))

In [13]:
compare(
        Agent(text=utils.bandits.neural.format("use_sep_nn,sep_components=True,128", "")),
        Agent(text=utils.bandits.neural)
)

(0.0029293476628126118,
 -26.4,
 28.06136133547337,
 0.1,
 array([594., 584., 583., 628., 557., 642., 618., 631., 544., 606.]),
 array([629., 596., 593., 654., 660., 659., 616., 656., 578., 610.]))

In [14]:
compare(
        Agent(text=utils.bandits.neural.format("use_sep_nn,sep_components=True,32", "")),
        Agent(text=utils.bandits.neural)
)

(0.9271675686005258,
 -1.0,
 34.59479729670345,
 0.4,
 array([681., 619., 621., 579., 640., 618., 558., 622., 707., 592.]),
 array([670., 662., 645., 616., 657., 632., 573., 613., 653., 526.]))

In [15]:
compare(
        Agent(text=utils.bandits.neural.format("use_sep_nn,sep_components=True,64", "")),
        Agent(text=utils.bandits.neural)
)

(0.0187255584503912,
 -22.4,
 30.130383336426373,
 0.3,
 array([621., 633., 594., 642., 675., 574., 615., 605., 624., 587.]),
 array([649., 663., 617., 684., 665., 633., 598., 682., 608., 595.]))

In [23]:
# compare(
#         Agent(text=utils.bandits.neural.format("use_sep_nn=True", "")),
#         Agent(text=utils.bandits.neural),T=20
# )

In [68]:
from importlib import reload
import utils
reload(utils)
reload(utils.bandits)
from utils.bandits import exact_gittins

In [81]:
a = Agent(text=utils.bandits.neural.format("use_sep_nn,lr,hidden=True,1e-3,5", ""))
b = Agent(text=utils.bandits.neural)
init_agent(a)
init_agent(b)
a.file,b.file

('tmp/b_0.5128946492173845.py', 'tmp/b_0.27847701490698573.py')

In [17]:
from kaggle_environments import make

In [82]:
compare(Agent(text=gittins_bb_delta), Agent(text=gittins_bb_delta),T=5)

(0.012351396701868866,
 14.8,
 13.22724461102916,
 0.8,
 array([612., 655., 612., 594., 632.]),
 array([603., 663., 582., 573., 610.]))

In [83]:
utils.bandits.compare(a, b, first_is_sep_nn=True, verbose=True, T=5)

layer_1_0_-0.694,0.444,-0.282,0.393,0.097,-0.654,0.433,-0.683,0.698,-0.378,-0.103,0.742,-0.109,0.519,-0.011,-0.351,0.171,0.062,0.217,-0.207,-0.354,0.277,-0.623,0.552,-0.435,0.120,0.099,-0.445,-0.008,-0.284,0.209,0.587,-0.557,-0.229,-0.521,-0.174,0.296,0.032,0.310,-0.370,-0.030,0.767,-0.076,0.255,-0.593,0.095,0.322,-0.190,-0.035,-0.665,-0.530,0.496,-0.053,-0.035,0.002,-0.140,0.388,0.056,-0.051,-0.476,-0.419,0.452,-0.643,0.660,0.042,-0.566,0.058,-0.342,0.263,-0.689,0.060,-0.261,-0.105,0.417,0.012,-0.355,0.342,0.265,0.210,0.239,-0.137,0.190,-0.202,0.196,0.278,0.008,0.281,-0.437,-0.036,-0.213,0.146,0.156,0.055,0.016,-0.180,0.292,0.315,-0.246,0.391,-0.325,0.230,-0.224,0.135,0.470,0.104,-0.228,0.331,0.129,-0.139,-0.180,0.261,0.284,-0.130,0.014,-0.455,-0.043,0.395,0.349,-0.138,-0.302,-0.307,0.096,0.017,-0.219,-0.438,-0.068,-0.130,0.180,0.189,0.191,-0.046,0.340,0.287,0.344,0.296,-0.141,-0.225,-0.186,0.422,0.152,-0.291,-0.145,0.221,-0.067,-0.308$
layer_1_1_-0.874,-0.230,-0.098,0.730,-0.780$
lay

(0.6165458877659269,
 10.0,
 44.654227123532216,
 0.6,
 'tmp/b_0.26871844307646753.py',
 'tmp/b_0.27847701490698573.py',
 array([642., 651., 594., 653., 684.]),
 array([694., 609., 630., 598., 643.]))

In [84]:
bench(Agent(file='tmp/b_0.26871844307646753.py'))

layer_1_0_-0.450,0.327,-0.026,0.157,0.361,-0.608,0.582,-0.611,0.662,-0.327,-0.004,0.802,0.010,0.422,0.096,-0.074,0.006,0.357,-0.054,0.070,-0.150,0.176,-0.401,0.345,-0.250,0.233,0.102,-0.318,-0.127,-0.192,0.538,0.333,-0.211,-0.547,-0.162,0.026,0.206,0.249,0.106,-0.152,0.004,0.871,-0.027,0.218,-0.566,0.127,0.429,-0.143,-0.069,-0.644,-0.324,0.406,0.171,-0.239,0.202,0.229,0.063,0.428,-0.411,-0.055,-0.557,0.730,-0.773,0.776,-0.084,-0.558,0.169,-0.333,0.255,-0.651,0.042,-0.233,-0.124,0.432,0.009,-0.354,0.345,0.254,0.213,0.246,-0.213,0.264,-0.280,0.270,0.206,-0.070,0.350,-0.521,0.036,-0.292,0.096,0.196,-0.008,0.083,-0.228,0.189,0.418,-0.365,0.487,-0.428,0.311,-0.237,0.178,0.425,0.192,-0.309,0.430,0.022,-0.053,-0.240,0.309,0.256,-0.114,-0.007,-0.378,-0.084,0.468,0.284,-0.082,-0.321,-0.444,0.222,-0.129,-0.087,-0.574,-0.194,-0.020,0.034,0.296,0.052,-0.158,0.434,0.153,0.443,0.169,-0.230,-0.145,-0.275,0.493,0.050,-0.428,-0.036,0.068,0.059,-0.463$
layer_1_1_-0.842,-0.121,-0.051,0.696,-0.759$
layer_

[['gittins', (0.00011465952292291106, 25.3, 20.741504284887345)],
 ['gittins_with_random', (0.0001775074569136045, 33.5, 28.256857574755195)],
 ['gittins_with_count_my', (0.7593656757444803, 3.0, 30.97095413447897)],
 ['gittins_with_count_rival_drift',
  (4.0854311366045265e-07, 54.9, 34.27666844954451)],
 ['gittins_with_my_and_count_rival_drift',
  (0.00873473842722055, 17.7, 21.345022839060164)],
 ['gittins_bb', (0.162369742926889, 13.5, 30.55568686840471)],
 ['gittins_bb_delta', (0.5797469292829708, 6.0, 34.26368339802363)],
 ['softmax_ucb', (5.582048149272401e-10, 49.7, 25.341862599264484)],
 ['multiarmed_bandit_agent', (0.18992243414746535, 15.5, 37.39318119657647)],
 ['upper_confidence', (3.411021341066924e-66, 137.2, 25.245989780557228)],
 ['ucb_decay', (0.0004195065502233244, 30.2, 27.073234014428348)],
 ['bayesian_ucb', (0.5180231760160137, -7.5, 36.69127961791467)],
 ['thompson', (0.2538623747418146, 6.9, 19.123022773609826)],
 ['neural', (0.5327811517733443, -4.9, 24.8412962

In [74]:
base_path = '/Users/sergmiller/Documents/my/bandits'
known_agents = [
        ('gittins', Agent(text=utils.bandits.gittins)),
        ('gittins_with_random', Agent(text=utils.bandits.gittins_with_random)),
        ('gittins_with_count_my', Agent(text=utils.bandits.gittins_with_count_my)),
        ('gittins_with_count_rival_drift', Agent(text=utils.bandits.gittins_with_count_rival_drift)),
        ('gittins_with_my_and_count_rival_drift', Agent(text=utils.bandits.gittins_with_my_and_count_rival_drift)),
        ('gittins_bb', Agent(text=utils.bandits.gittins_bb)),
        ('gittins_bb_delta', Agent(text=utils.bandits.gittins_bb_delta)),
        ('softmax_ucb', Agent(file=base_path + '/kernels/softmax_ucb.py')),
        ('multiarmed_bandit_agent', Agent(file=base_path + '/kernels/multiarmed_bandit_agent.py')),
        ('upper_confidence', Agent(file=base_path + '/kernels/upper_confidence.py')),
        ('ucb_decay', Agent(file=base_path + '/kernels/ucb_decay.py')),
        ('bayesian_ucb', Agent(file=base_path + '/kernels/bayesian_ucb.py')),
        ('thompson', Agent(file=base_path + '/kernels/thompson.py')),
        ('neural', Agent(text=utils.bandits.neural)),
#         ('max_likelihood', Agent(file=path + 'kernels/max_likelihood.py')), works too long
        ('optimized_ucb', Agent(file=base_path + '/kernels/optimized_ucb.py')),
        ('exact_gittins', Agent(text=utils.bandits.exact_gittins)),
    ]

In [75]:
for n,a in known_agents:
    init_agent(a)
    print(n, a.file)

gittins tmp/b_0.9119244951369558.py
gittins_with_random tmp/b_0.7033635689233245.py
gittins_with_count_my tmp/b_0.541779649962201.py
gittins_with_count_rival_drift tmp/b_0.10143611548611686.py
gittins_with_my_and_count_rival_drift tmp/b_0.3490039729438965.py
gittins_bb tmp/b_0.9703834174439596.py
gittins_bb_delta tmp/b_0.32177657873570265.py
softmax_ucb /Users/sergmiller/Documents/my/bandits/kernels/softmax_ucb.py
multiarmed_bandit_agent /Users/sergmiller/Documents/my/bandits/kernels/multiarmed_bandit_agent.py
upper_confidence /Users/sergmiller/Documents/my/bandits/kernels/upper_confidence.py
ucb_decay /Users/sergmiller/Documents/my/bandits/kernels/ucb_decay.py
bayesian_ucb /Users/sergmiller/Documents/my/bandits/kernels/bayesian_ucb.py
thompson /Users/sergmiller/Documents/my/bandits/kernels/thompson.py
neural tmp/b_0.3069883745398595.py
optimized_ucb /Users/sergmiller/Documents/my/bandits/kernels/optimized_ucb.py
exact_gittins tmp/b_0.6823684138662427.py


In [5]:
# with frozen weights -> bad
bench(Agent(file='tmp/b_0.26871844307646753.py'))

['gittins', (0.0048683409089503495, 22.9, 25.719447894540817)]
['gittins_with_random', (0.008171181845754391, 26.5, 31.683591968083416)]
['gittins_with_count_my', (0.290446686019758, 9.0, 26.922109872742144)]
['gittins_with_count_rival_drift', (4.645172164700792e-09, 44.3, 23.90836673635403)]
['gittins_with_my_and_count_rival_drift', (0.6559138714636232, 3.5, 24.840491138461815)]
['gittins_bb', (0.5250439976740156, 6.7, 33.33481663366397)]
['gittins_bb_delta', (0.8297230503174098, 2.6, 38.23140070674889)]
['softmax_ucb', (0.00010895418130192162, 31.1, 25.414366016094124)]
['multiarmed_bandit_agent', (0.6728787999877004, -4.0, 29.95997329771841)]
['upper_confidence', (8.38220128991495e-39, 147.5, 35.80013966453204)]
['ucb_decay', (0.0069967676310094505, 23.7, 27.788666754632185)]
['bayesian_ucb', (0.0010196646956593498, -23.0, 22.140460699813815)]
['thompson', (0.38928686675610547, -9.1, 33.425888170697874)]
['neural', (0.41457591961551055, 11.0, 42.635665821000146)]
<string>:82: Runtim

[['gittins', (0.0048683409089503495, 22.9, 25.719447894540817)],
 ['gittins_with_random', (0.008171181845754391, 26.5, 31.683591968083416)],
 ['gittins_with_count_my', (0.290446686019758, 9.0, 26.922109872742144)],
 ['gittins_with_count_rival_drift',
  (4.645172164700792e-09, 44.3, 23.90836673635403)],
 ['gittins_with_my_and_count_rival_drift',
  (0.6559138714636232, 3.5, 24.840491138461815)],
 ['gittins_bb', (0.5250439976740156, 6.7, 33.33481663366397)],
 ['gittins_bb_delta', (0.8297230503174098, 2.6, 38.23140070674889)],
 ['softmax_ucb', (0.00010895418130192162, 31.1, 25.414366016094124)],
 ['multiarmed_bandit_agent', (0.6728787999877004, -4.0, 29.95997329771841)],
 ['upper_confidence', (8.38220128991495e-39, 147.5, 35.80013966453204)],
 ['ucb_decay', (0.0069967676310094505, 23.7, 27.788666754632185)],
 ['bayesian_ucb', (0.0010196646956593498, -23.0, 22.140460699813815)],
 ['thompson', (0.38928686675610547, -9.1, 33.425888170697874)],
 ['neural', (0.41457591961551055, 11.0, 42.635665

In [64]:
from importlib import reload
import utils
reload(utils)
reload(utils.bandits)
from utils.bandits import exact_gittins

In [11]:
custom = utils.bandits.neural_with_new_feature_vec.format("{}", "3", "prior_estimations")

In [12]:
a = Agent(text=custom)
init_agent(a)
a.file

'tmp/b_0.23596647675733917.py'

In [9]:
tuple([1,2,3])

(1, 2, 3)

In [74]:
a = Agent(text=utils.bandits.neural.format("use_sep_nn,lr,hidden=True,1e-3,5", ""))
# b = Agent(text=utils.bandits.neural)
b = Agent(file='kernels/bayesian_ucb.py')
init_agent(a)
init_agent(b)
a.file,b.file

('tmp/b_0.484535000365256.py', 'kernels/bayesian_ucb.py')

In [75]:
b = Agent(file='kernels/bayesian_ucb.py')
utils.bandits.compare(a, b, first_is_sep_nn=True, verbose=True, T=20)

layer_1_0_0.15,-0.03,0.41,-0.18,0.52,0.17,0.19,0.14,0.34,0.31,0.50,0.09,0.13,-0.36,0.43,0.00,0.47,0.53,-0.22,-0.00,0.06,0.43,0.34,0.00,-0.15,-0.00,0.02,0.17,-0.14,0.11,0.19,-0.02,0.35,0.20,-0.12,0.16,0.51,0.17,0.01,-0.19,-0.00,0.45,-0.04,-0.34,0.09,-0.33,-0.07,0.01,-0.21,0.06,-0.28,-0.04,0.38,-0.08,0.16,0.37,-0.19,0.02,-0.01,-0.02,0.11,0.07,-0.03,-0.00,0.15,-0.01,-0.23,0.36,0.04,-0.02,-0.26,-0.12,0.19,0.39,0.09,0.19,-0.07,0.34,-0.25,-0.35,0.29,0.37,0.11,0.31,-0.15,-0.27,-0.22,0.28,-0.17,0.12,-0.01,0.15,0.07,-0.29,0.25,0.13,0.14,0.15,0.33,-0.14,-0.24,-0.14,0.23,0.01,0.08,0.03,0.05,-0.21,0.02,-0.26,0.27,-0.30,-0.10,0.03,-0.33,-0.31,0.27,0.06,0.26,0.30,-0.19,0.17,-0.28,-0.14,0.08,-0.17,-0.13,-0.16,0.28,-0.20,-0.20,-0.24,0.30,0.04,0.11,0.12,0.06,0.32,-0.17,-0.15,-0.24,0.07,0.39,-0.27,-0.25,-0.05,0.44,0.40,-0.31,-0.05,-0.27,0.11,0.14,0.28,0.52,0.21,0.42,0.21,-0.33,0.48,0.30,0.15,-0.10,0.04,-0.10,0.05,-0.12,0.38,0.06,0.49,-0.14,0.34,-0.11,0.08,0.44,0.19,0.16,0.17,-0.11,0.27$
layer_1_1_-0.59,

(0.6262134467193456,
 -3.5,
 32.13642792844283,
 0.5,
 'tmp/b_0.11733052128441013.py',
 'kernels/bayesian_ucb.py',
 array([532., 638., 597., 607., 715., 634., 648., 566., 609., 602., 643.,
        651., 562., 667., 645., 575., 565., 596., 641., 669.]),
 array([590., 611., 649., 601., 716., 642., 608., 605., 590., 596., 613.,
        615., 559., 683., 696., 589., 524., 651., 647., 647.]))

In [76]:
b = Agent(file='templates/neural.py')
utils.bandits.compare(Agent(file='tmp/b_0.11733052128441013.py'), b, verbose=True, T=10)

layer_1_0_-0.22,-0.63,-0.32,-0.27,-0.01,0.33,0.00,-0.14,0.70,0.31,0.48,-0.21,-0.28,-0.16,0.26,0.07,0.30,0.28,0.01,-0.02,0.15,0.29,0.13,0.24,-0.13,-0.55,-0.77,-0.73,-0.33,-0.63,1.08,0.59,0.90,1.38,0.64,0.70,0.78,0.36,0.72,0.26,-0.66,-0.37,-0.96,-0.72,-0.70,-0.92,-0.80,-0.82,-0.59,-0.62,-0.50,-0.46,-0.09,-0.15,-0.13,0.01,-0.74,-0.62,-0.14,-0.47,-0.05,-0.31,-0.50,0.01,-0.10,-0.45,-0.88,-0.36,-0.24,-0.59,-0.12,0.00,0.30,0.48,0.24,0.23,-0.02,0.38,-0.22,-0.27,0.42,0.57,0.33,0.36,0.07,-0.15,-0.11,0.38,0.10,0.26,0.07,0.20,0.16,-0.21,0.32,-0.21,-0.28,-0.37,0.17,-0.54,-0.42,-0.43,-0.13,0.11,-0.17,-0.01,-0.08,-0.39,0.23,-0.34,0.32,-0.28,-0.10,0.21,-0.23,-0.17,0.36,0.14,0.25,0.42,-0.01,0.35,-0.13,0.18,0.31,0.07,0.17,0.12,0.47,0.11,-0.16,-0.20,0.32,0.07,0.20,0.37,0.24,0.49,0.18,0.09,-0.17,0.07,0.40,-0.13,-0.19,-0.06,0.22,0.10,-0.22,-0.14,-0.42,-0.26,-0.31,0.26,0.30,0.27,0.25,-0.05,-0.18,0.47,0.40,0.01,-0.29,0.27,-0.07,-0.33,-0.70,-0.27,-0.08,-0.03,-0.54,-0.25,-0.78,-0.06,-0.08,-0.24,-0.45,-0.53,-0.

(0.27766778675631665,
 -10.6,
 30.87782375751244,
 0.2,
 'tmp/b_0.11733052128441013.py',
 'templates/neural.py',
 array([665., 620., 598., 580., 649., 619., 639., 610., 638., 611.]),
 array([667., 570., 643., 590., 606., 654., 671., 639., 657., 638.]))

In [57]:
b = Agent(text=utils.bandits.neural)
utils.bandits.compare(a, b, first_is_sep_nn=True, verbose=True, T=5)

layer_1_0_0.05,-0.02,-0.05,0.05,-0.28,0.20,-0.44,0.15,0.47,-0.19,0.35,0.17,-0.30,-0.04,0.26,-0.26,-0.25,0.30,0.06,-0.19,0.19,0.06,-0.05,-0.15,0.08,-0.17,-0.22,0.26,0.30,-0.33,0.21,0.09,0.21,0.48,0.01,0.04,-0.06,0.11,0.47,-0.12,0.11,0.22,0.07,0.43,0.19,0.36,0.03,0.08,-0.18,0.08,0.10,0.27,-0.42,0.34,-0.11,-0.08,-0.04,0.08,-0.08,0.07,0.10,-0.22,0.19,0.07,-0.34,-0.37,-0.37,0.12,-0.08,0.26,-0.12,0.06,-0.24,-0.10,0.08,0.01,-0.21,0.04,0.42,-0.19,-0.14,0.21,0.33,0.31,-0.31,-0.12,-0.25,0.14,-0.30,-0.04,0.29,0.21,-0.09,0.20,0.10,0.36,-0.10,0.29,-0.29,0.19,-0.19,0.07,0.03,0.06,0.32,-0.36,-0.23,-0.02,0.26,0.25,-0.23,-0.29,0.25,0.11,-0.27,0.07,0.26,-0.29,-0.27,-0.21,-0.13,-0.08,0.12,0.07,0.32,-0.25,-0.28,-0.14,0.04,-0.29,-0.34,-0.01,-0.01,0.14,0.28,-0.27,0.14,0.25,-0.28,0.01,-0.17,0.19,0.16,-0.08,-0.40,0.31,0.07,-0.22,0.22,0.21,-0.19,-0.05,-0.27,0.34,0.14,-0.22,0.17,-0.28,-0.00,0.00,-0.26,-0.06,-0.30,0.32,0.24,-0.37,0.29,0.03,0.40,-0.25,-0.33,0.08,-0.42,0.46,-0.03,-0.39,0.13,0.20,0.25,0.13$
layer_1

(0.340687435749466,
 9.2,
 21.590738755308955,
 0.8,
 'tmp/b_0.6460890804954242.py',
 'tmp/b_0.9433293439410158.py',
 array([668., 626., 608., 631., 614.]),
 array([649., 602., 581., 623., 646.]))

In [59]:
utils.bandits.compare(Agent(file='tmp/b_0.6460890804954242.py'),
                      Agent(text=utils.bandits.neural), T=20, verbose=True)

layer_1_0_0.06,0.16,-0.01,-0.11,-0.06,0.39,-0.46,0.12,0.49,-0.15,0.46,0.21,-0.32,-0.09,0.35,-0.13,-0.26,0.23,0.06,-0.15,0.32,0.05,-0.12,-0.15,0.11,-0.22,0.07,0.42,0.05,-0.03,0.63,-0.24,0.26,0.76,-0.26,0.30,-0.17,0.29,0.53,-0.16,0.03,0.45,0.09,0.21,0.45,0.31,0.16,0.02,-0.35,0.27,0.14,0.33,-0.49,0.26,0.01,-0.07,0.07,0.05,-0.21,0.24,0.14,-0.08,0.22,-0.06,-0.16,-0.40,-0.22,0.10,-0.25,0.45,-0.17,0.10,-0.33,-0.15,0.14,-0.00,-0.25,-0.00,0.44,-0.21,-0.06,0.14,0.28,0.35,-0.36,-0.01,-0.40,0.06,-0.17,-0.19,0.40,0.09,-0.20,0.30,-0.00,0.32,0.08,0.27,-0.43,0.35,-0.15,0.20,-0.02,-0.05,0.44,-0.30,-0.18,-0.07,0.21,0.29,-0.01,-0.39,0.24,0.20,-0.38,0.26,0.16,-0.26,-0.18,-0.31,-0.10,-0.14,0.09,0.09,0.28,-0.11,-0.44,-0.23,0.17,-0.45,-0.22,-0.06,-0.04,0.17,0.22,-0.17,-0.02,0.14,-0.18,-0.12,-0.04,0.07,0.13,0.01,-0.51,0.42,0.07,-0.31,0.20,0.27,-0.13,-0.01,-0.35,0.28,0.22,-0.10,0.16,-0.38,-0.01,0.04,-0.13,-0.09,-0.40,0.32,0.26,-0.42,0.48,0.04,0.22,-0.00,-0.38,0.26,-0.41,0.29,0.19,-0.44,0.32,0.21,0.07,0.37$
lay

(0.06126984536348638,
 12.1,
 28.91349166046882,
 0.65,
 'tmp/b_0.6460890804954242.py',
 'tmp/b_0.007303229434929226.py',
 array([632., 703., 617., 621., 650., 626., 690., 694., 636., 594., 592.,
        630., 648., 699., 539., 642., 663., 650., 667., 640.]),
 array([593., 645., 612., 636., 623., 595., 685., 640., 607., 609., 588.,
        568., 697., 710., 532., 616., 671., 680., 640., 644.]))

In [61]:
utils.bandits.compare(Agent(file='tmp/b_0.6460890804954242.py'),
                      Agent(file='kernels/bayesian_ucb.py'), T=20, verbose=True)

layer_1_0_0.09,0.16,-0.05,-0.10,-0.08,0.38,-0.42,0.12,0.47,-0.13,0.45,0.24,-0.34,-0.10,0.37,-0.15,-0.21,0.23,0.03,-0.12,0.30,0.10,-0.11,-0.18,0.14,-0.17,0.02,0.34,0.10,-0.09,0.64,-0.23,0.20,0.76,-0.26,0.31,-0.17,0.26,0.55,-0.17,0.05,0.44,0.07,0.22,0.43,0.29,0.20,0.04,-0.37,0.30,0.13,0.37,-0.48,0.24,0.03,-0.06,0.08,0.04,-0.20,0.23,0.15,-0.06,0.21,-0.07,-0.16,-0.39,-0.21,0.10,-0.25,0.45,-0.13,0.07,-0.35,-0.12,0.10,0.03,-0.26,0.01,0.46,-0.23,-0.06,0.15,0.28,0.35,-0.36,-0.00,-0.38,0.10,-0.19,-0.18,0.26,0.22,-0.08,0.18,0.12,0.29,0.11,0.26,-0.45,0.38,-0.18,0.23,-0.01,-0.08,0.47,-0.34,-0.12,-0.04,0.16,0.34,-0.10,-0.29,0.26,0.12,-0.27,0.22,0.21,-0.28,-0.22,-0.27,-0.11,-0.13,0.13,0.08,0.30,-0.10,-0.42,-0.15,0.15,-0.44,-0.21,-0.06,-0.03,0.17,0.23,-0.12,-0.05,0.16,-0.16,-0.14,-0.03,0.09,0.20,-0.00,-0.49,0.39,0.12,-0.30,0.17,0.31,-0.14,0.03,-0.34,0.26,0.24,-0.13,0.21,-0.36,-0.04,0.08,-0.16,-0.03,-0.38,0.29,0.30,-0.40,0.47,0.01,0.24,-0.03,-0.36,0.25,-0.44,0.30,0.17,-0.42,0.31,0.18,0.09,0.35$
layer_

(0.07152473413708584,
 -9.55,
 23.69910335856612,
 0.3,
 'tmp/b_0.6460890804954242.py',
 'kernels/bayesian_ucb.py',
 array([687., 598., 660., 612., 606., 655., 639., 623., 641., 634., 679.,
        604., 640., 590., 583., 733., 571., 645., 658., 557.]),
 array([675., 624., 632., 616., 615., 669., 623., 648., 620., 645., 654.,
        633., 652., 641., 630., 703., 597., 675., 672., 582.]))

In [67]:
utils.bandits.compare(Agent(text=utils.bandits.neural.format("{}", "baseline=bucb")),
                      Agent(text=utils.bandits.neural), T=10, verbose=True)

609.0 621.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
576.0 610.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
692.0 685.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
607.0 639.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
601.0 671.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
567.0 630.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
682.0 632.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
692.0 746.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
586.0 634.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py
581.0 643.0 tmp/b_0.8652729767510375.py tmp/b_0.2096976143387005.py


(0.004765008315775955,
 -31.8,
 35.628078814328454,
 0.2,
 'tmp/b_0.8652729767510375.py',
 'tmp/b_0.2096976143387005.py',
 array([609., 576., 692., 607., 601., 567., 682., 692., 586., 581.]),
 array([621., 610., 685., 639., 671., 630., 632., 746., 634., 643.]))

In [68]:
utils.bandits.compare(Agent(text=utils.bandits.neural.format("{}", "baseline=bucb")),
                      Agent(file='kernels/bayesian_ucb.py'), T=10, verbose=True)

559.0 635.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
652.0 673.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
491.0 625.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
563.0 582.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
633.0 693.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
606.0 660.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
617.0 692.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
562.0 643.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
601.0 628.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py
548.0 634.0 tmp/b_0.021604991878027002.py kernels/bayesian_ucb.py


(2.5715278765020693e-09,
 -63.3,
 33.60372003216311,
 0.0,
 'tmp/b_0.021604991878027002.py',
 'kernels/bayesian_ucb.py',
 array([559., 652., 491., 563., 633., 606., 617., 562., 601., 548.]),
 array([635., 673., 625., 582., 693., 660., 692., 643., 628., 634.]))